# [2.4] - RLHF


Colab: [exercises](https://colab.research.google.com/drive/13TDGeRdUcZ30nlfkN_PAxQxe8oX2x49u?usp=sharing) | [solutions](https://colab.research.google.com/drive/1KEXcflwuTGxf6JkAWdDCveXldDs6qgfw?usp=sharing)

[Streamlit page](https://arena3-chapter2-rl.streamlit.app/[2.4]_RLHF)

Please send any problems / bugs on the `#errata` channel in the [Slack group](https://join.slack.com/t/arena-la82367/shared_invite/zt-1uvoagohe-JUv9xB7Vr143pdx1UBPrzQ), and ask any questions on the dedicated channels for this chapter of material.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/shoggoth.png" width="350">


# Introduction


This section is designed to take you through a full implementation of RLHF (Reinforcement Learning from Human Feedback). Much of this follows on directly from the PPO implementation from yesterday, with only a few minor adjustments and new concepts. You'll (hopefully) be pleased to learn that we're disposing of OpenAI's gym environment for this final day of exercises, and instead going back to our week 1 roots with TransformerLens!

We'll start by discussing how the RL setting we've used for tasks like CartPole and Atari fits into the world of autoregressive transformer language models. We'll then go through standard parts of the PPO setup (e.g. objective function, memory buffer, rollout and learning phases) and show how to adapt them for our transformer. Finally, we'll put everything together into a `RLHFTrainer` class, and perform RLHF on our transformer!

> **Note - these exercises assume you're running on an A100 (either a virtual machine or Colab Pro+).** If you're running on a less powerful machine e.g. A10, we recommend changing `"gpt2-medium"` to `"gpt2-small"`. You might also want to drop the batch size, if you're running out of memory.

## Content & Learning Objectives



#### 1️⃣ RLHF on transformer language models

Most of the exercises today build towards the implementation of the `RLHFTrainer` class, similar to how DQN and PPO have worked these last few days.

> ##### Learning objectives
>
> - Understand how the RL agent / action / environment paradigm works in the context of autoregressive transformer models
> - Understand how the RLHF algorithm works, and how it fits on top of PPO
> - Learn about value heads, and how they can be used to turn transformers into actor & critic networks with shared architectures
> - Write a full RLHF training loop, and use it to train your transformer with the "maximize output of periods" reward function
> - Observe and understand the instances of mode collapse that occur when training with this reward function
> - Experiment with different reward functions, and training parameters

#### 2️⃣ Bonus

This section offers some suggested ways to extend the core RLHF exercises.

> #### Learning objectives
>  
> - Improve your RLHF implementation via techniques like differential learning rates, frozen layers, or adaptive KL penalties
> - Perform some exploratory mechanistic interpretability on RLHF'd models
> - Learn about the trlX library, which is designed to train transformers via RLHF in a way which abstracts away many of the low-level details

## Reading

- [Illustrating Reinforcement Learning from Human Feedback (RLHF)](https://huggingface.co/blog/rlhf) (~10 minutes)
    - An accessible and mostly non-technical introduction to RLHF, which discusses it in context of the full pipeline for training autoregressive transformer language models (starting with pretraining, which is what we did in the first day of last week).
- [RLHF+ChatGPT: What you must know](https://www.youtube.com/watch?v=PBH2nImUM5c) (~5 minutes)
    - The first half of this video provides a high-level overview of RLHF, discussing things like mode collapse, and relates this to the [shoggoth meme](https://i.kym-cdn.com/photos/images/original/002/546/572/bd3.png) that many of you have likely seen!

## Setup (don't read, just run!)

In [1]:
try:
    import google.colab # type: ignore
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys
chapter = "chapter2_rl"
repo = "ARENA_3.0"

if IN_COLAB:

    # Code to download the necessary files (e.g. solutions, test funcs)
    import os, sys
    if not os.path.exists(f"/content/{chapter}"):

        # Install packages
        %pip install transformer_lens
        %pip install git+https://github.com/callummcdougall/eindex.git
        %pip install wandb

        # Clone repo
        os.system(f"wget https://github.com/callummcdougall/{repo}/archive/refs/heads/main.zip")
        os.system(f"unzip /content/main.zip '{repo}-main/{chapter}/exercises/*'")
        sys.path.append(f"/content/{repo}-main/{chapter}/exercises")
        os.remove("/content/main.zip")
        os.rename(f"{repo}-main/{chapter}", chapter)
        os.rmdir(f"{repo}-main")

    os.chdir(f"{chapter}/exercises")
else:
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import torch as t
import torch.nn as nn
from torch import Tensor
import wandb
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer
from typing import List, Optional, Tuple, Union, Dict, Any, Callable
import einops
from jaxtyping import Float, Int
import os
import sys
from pathlib import Path
from rich import print as rprint
from rich.table import Table
from eindex import eindex
from dataclasses import dataclass
import numpy as np
import time
from functools import partial

# Make sure exercises are in the path
chapter = "chapter2_rl"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = exercises_dir / "part4_rlhf"
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

import part4_rlhf.tests as tests
import part4_rlhf.solutions as solutions

device = t.device('cuda' if t.cuda.is_available() else 'cpu')

# 1️⃣ RLHF on transformer language models

## The "transformer environment"

We'll start by discussing how we apply the reinforcement learning framework of states/actions/rewards to the setting of autoregressive language modelling.



### States, actions and episodes

Our actor is an autoregressive language model. The actions $a_t$ are the tokens generated by the model (i.e. the action space is the model's vocabulary). The states $s_t$ are **the entire sequence up to that point** (not just the most recent token). In other words, given a state $s_t$ (sequence) and action $a_t$ (token generation), our new state is the concatenation which we'll denote as $s_{t+1} = [s_t \; a_t]$.

Each episode is a fixed length (i.e. all our sampled outputs will have the same number of tokens generated from them). Each episode starts with an initial "prefix prompt", which is chosen before the start of training.



### Rewards and value functions

The reward $r_t$ is a function of the sequence $s_t$. Sometimes it will be a very simple function like the sum of periods `.` in the sequence, other times it'll get a bit more complicated (e.g. using a text classification model to estimate the sentiment of a sequence - we'll do this later!).

In our case, we'll only evaluate the reward at the end of the episode. This means we don't really have a concept of discount factors here - the reward only comes once, and as soon as it comes our episode terminates.

The value function $V(s_t)$ is an estimate of the expected sum of future rewards (up to the end of the episode), which in this case means it's an estimate of what the reward will be once we get to the end of the sequence. We'll be adding a value head to our transformer model to estimate this value function (more on this later).



### ~~Generalized~~ Advantage Estimation

We won't be using the GAE formula today for computing advantages, we'll just be directly computing it via $A(s_t, a_t) = Q(s_t, a_t) - V(s_t)$, where $a_t$ is the value which was actually taken and $Q(s_t, a_t)$ is the critic's estimate of the value function at this new state $s_{t+1} = [s_t \; a_t]$.

We can get away with this because our setup has pretty low variance when it comes to the advantage of particular actions. GAE is most helpful when it reduces variance in the advantage estimation (it does this at the cost of introducing more bias from including future value function estimates), and so it's especially useful when our environment is one with high variability when the advantage (and optimal policy) changes significantly between steps. But this doesn't really apply to us, since every action just adds a single token onto our sequence.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/transformer-rl-state.png" width="700">

## RLHF Setup

With this context in mind, we're now ready to look at the full RLHF setup we'll be using:

<img src="https://pbs.twimg.com/media/FkLOrrPWYAAiFLF.jpg:large" width="700">

Our autoregressive transformer model (we'll be using GPT2-Small) is the actor, and its value head will play the role of the critic. We follow the standard PPO setup:

- In **rollout phase**, the actor generates a bunch of sequences all starting from the prefix prompt. We compute advantage estimates using the critic network (value head) and store the experiences in memory.
- In **learning phase**, we sample from these generated experiences (i.e. from a bunch of generated sequences of different lengths, some of which might be prefixes of each other). We compute our loss function (which is the sum of the same 3 terms as yesterday) and perform a gradient step with respect to it.

The only new element is the **KL prediction shift penalty**. This is a penalty we add to our overall loss function to stop the transformer from diverging too much from its initial distribution. We want to make our transformer maximize reward, but not in a way which causes it to become completely incoherent!

Note that we compute $D_{KL}(\pi_{PPO} || \pi_{base})$, not the other way around. This is because we want to penalize our new model for generating outputs which would be extremely unlikely under the old model (recall that $D_{KL}(P || Q)$ blows up when some probability $Q_i$ is very small and $P_i$ is not small). For example, when we train our model to maximize the number periods it outputs, we want to observe behaviour like shorter sentences and use of initials, not crazy mode-collapse behaviour like only outputting periods! On the other hand, we don't really care about the other kind of failure mode: if our original model had a very small chance of generating offensive hate speech, we definitely don't want to penalize our new model for completely eradicating this behaviour!

<!-- An alternative perspective can be found from [this post](https://www.lesswrong.com/posts/no5jDTut5Byjqb4j5/six-and-a-half-intuitions-for-kl-divergence) - the KL divergence $D_{KL}(P || Q)$ is large when the observations $P$ give you a lot of evidence that your hypothesis $Q$ is false. We want to make sure that the original (probably coherent and sensible) model $Q$ is still a good approximation for how $P$ behaves, i.e. it shouldn't be too obvious when we observe the outputs of $P$ that they've been generated by a different model. -->

## Value head

If you worked on the Atari exercises yesterday, then you'l be used to the idea of having shared architecture between our policy and value networks. Intuitively, this is because both networks need to learn some kind of high-level encoding of the important variables in the environment - they just do different things with this encoding.

This leads to the idea of a **value head**. A value head is basically just a simple classifier model which we stick to one of the policy network's internal activations. You can think of this as a kind of feature extraction. When it comes to transformer models, we usually attach our value head to **the value of the residual stream at the very last layer, after layernorm but before unembedding**. Recall the key idea of **residual stream as output accumulation** - by the very last layer, it contains the most context about the overall sequence.\*

\*Technically this might not always be true, since there is some evidence that components of a transformer erase information in order to write different information to the residual stream. However, in practice we usually find that the residual stream at the last layer is the most useful for downstream tasks.

How do we implement this? Before you read further down, try to think about how you might implement this yourself, i.e. how you could extend the functionality of your `HookedTransformer` model by adding a value head.

<details>
<summary>Hint</summary>

Our solution uses hook functions.

</details>

<details>
<summary>Answer - what we'll be doing in these exercises.</summary>

One method would be to directly edit the model by replacing its modules with different ones. But this is a bit awkward, because we have to also change modules which are downstream of the value head to make sure that they're only taking the residual stream as input (not the value head's output), etc.

A different method, which is what we'll be using in these exercises, is to use **hook functions**. We can attach a hook function to the residual stream at the final layer, and have it apply our value head to the residual stream values & store the output externally. Then we can use `model.run_with_hooks` to get our logits like normal, and fetch our value estimate from the external storage object.

We're used to using hook functions during inference mode to perform causal interventions or compute statistical functions of our activations, but they can also be used during training mode to perform computations which are part of the autograd's computational graph.

</details>

Engineering-wise, we can get a GPTwithvaluehead class by wrapping around the `HookedRootModule` (like in reference page) but adding a value head. This would be a nice way of introducing people to how the `HookedRootModule` works.

### Exercise - implement `TransformerWithValueHead`

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 15-25 minutes on this exercise.
```

Here is a diagram of your implementation.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/value-head-3.png" width="600">

First define `self.base_model` and `self.value_head` in your init step (reminder that you should use `HookedTransformer.from_pretrained` to load in a pretrained model). Then rewrite the `forward` method so that it outputs both the logits from a forward pass *and* the output of the value head.

The easiest and most direct way to get the output of the value head would be to **add a hook to the residual stream before the unembedding matrix, which computes the output of the value head and stores it externally (or as a class attribute).** You can review the material from section 1.2 if you don't remember how to use hooks, and you can refer to the diagram on the [reference page](https://arena3-chapter1-transformer-interp.streamlit.app/Reference_Page) for how to get the correct hook name.

Why do we need to add the hook after the layernorm? The answer is that the residual stream can often [grow in magnitude over time](https://www.lesswrong.com/posts/8mizBCm3dyc432nK8/residual-stream-norms-grow-exponentially-over-the-forward). Our rewards will be normalized (see later exercise), and so we want to make sure the outputs of our value head (which are estimates of the reward) also start off normalized.

In [49]:
# model.base_model.unembed = nn.Sequential()
# list(model.base_model.modules())[:-1]

In [42]:
class TransformerWithValueHead(nn.Module):
    '''
    Defines a GPT model with a value head (the latter taking the last hidden state as input,
    post-layernorm).

    The value head is a simple MLP with one hidden layer, and scalar output:

        Linear(d_model -> 4*d_model)
        ReLU
        Linear(4*d_model -> 1)

    All linear layers have biases.
    '''
    base_model: HookedTransformer
    value_head: nn.Sequential

    def __init__(self, base_model: str = "gpt2-medium"):
        super().__init__()
        self.base_model = HookedTransformer.from_pretrained("gpt2-medium")
        self.value_head = nn.Sequential(
          nn.Linear(self.base_model.cfg.d_model, 4 * self.base_model.cfg.d_model),
          nn.ReLU(),
          nn.Linear(4 * self.base_model.cfg.d_model, 1),
        )
        self.old_head = None
        def hook_fn(module, input, output):
          self.old_head = output
        self.base_model.ln_final.register_forward_hook(hook_fn)

    def forward(self, input_ids: Int[Tensor, "batch seq"]) -> Tuple[
        Float[Tensor, "batch seq d_vocab"],
        Int[Tensor, "batch seq"]
    ]:
        model_logits = self.base_model(input_ids)
        # print(f'{model_logits.shape=}')
        # print(f'{self.old_head.shape=}')
        ret = self.value_head(self.old_head).squeeze(-1)
        # print(f'{ret.shape=}')
        return model_logits, ret


# Define a reference model (we'll use this during RLHF)
model = TransformerWithValueHead().to(device)

# Test your value head's architecture
assert isinstance(model.base_model, HookedTransformer), "Your model should have a HookedTransformer as its `base_model` attribute."
assert isinstance(model.value_head, nn.Sequential), "Your model should have a `value_head` attribute that is a `nn.Sequential`."
d_model = model.base_model.cfg.d_model
assert len(model.value_head) == 3, "Your value head should be a `nn.Sequential` with 3 layers."
assert sum(p.numel() for p in model.value_head.parameters()) == (d_model+1)*4*d_model + (4*d_model+1), "Your value head should have the correct number of parameters."

# Test your class's forward pass
input_ids = t.randint(0, 1000, (1, 10)).to(device)
logits, values = model(input_ids)
assert logits.shape == (*input_ids.shape, model.base_model.cfg.d_vocab), "Your model's logits should have shape (batch, seq, d_vocab)."
assert values.shape == input_ids.shape, "Your model's value head should give you an output for every token in your input. Did you forget to squeeze the out_features=1 dim?"

print("All tests for `TransformerWithValueHead` passed!")

Loaded pretrained model gpt2-medium into HookedTransformer
All tests for `TransformerWithValueHead` passed!


## Sampling from a transformer

If you didn't go through the sampling exercises during the first day of last week, you might want to go back to them and work through the first few of them (this is not essential). Otherwise, here's a quick refresher:

- The simplest form of sampling is **greedy sampling**, where we autoregressively generate text by always choosing the most likely token at each step (i.e. argmaxing over logits), appending this to our sequence, and continuing.
- Most other forms of sampling are non-deterministic, i.e. they involve randomness. The most basic form of random sampling is choosing the next token according to the model's logit distribution.
- Other common refinements of this basic method are:
    - **Top-k sampling**, where we only consider the top-k most likely tokens at each step, and choose from these according to the model's logit distribution.
    - **Top-p sampling** (also called **nucleus sampling**), where we only consider the top-p most likely tokens at each step, and choose from these according to the model's logit distribution.

We've provided the model sampling code for you below, because there are a few non-obvious things to consider that are specific to our current situation. Make sure you completely understand this function before moving on to the next section.

We'll highlight a few things about this function:

- `.generate` is the standard method to autoregressively generate text. This works for TransformerLens slightly differently than for HuggingFace models (TransformerLens isn't primarily designed for text generation). In particular, it doesn't have features to efficiently generate multiple outputs for a single completion by using key-value caching. So rather than passing an argument into `generate` telling the model to generate `batch_size` outputs, we've instead just repeated `input_ids` multiple times across the batch dimension. This is a bit wasteful since we're repeating computation on the input sequence, but it's not a big problem because the input sequences we'll be using are usually very short.
    - As a bonus exercise later, we've suggested you write a version of the `generate` method which uses TransformerLens' key value caching (since TL does support caching behaviour, it just doesn't have features to use caching in `generate` to produce multiple sequences from a single completion).
- We've used `stop_at_eos = False`, to make sure that the model generates the full `gen_length` tokens rather than stopping early.

In [50]:
@t.no_grad()
def get_samples(base_model: HookedTransformer, prompt: str, batch_size: int, gen_len: int, temperature: float):
    '''
    Generates samples from the model, which will be fed into the reward model and evaluated.

    Inputs:
        gpt: the transformer to generate samples from (note we use gpt, not the model wrapper, cause we don't need value head)
        prompt: the initial prompt fed into the model
        batch_size: the number of samples to generate
        gen_len: the length of the generated samples (i.e. the number of *new* tokens to generate)

    Returns:
        sample_ids: the token ids of the generated samples (including initial prompt)
        samples: the generated samples (including initial prompt)
    '''
    # Make sure we've passed in the base model (the bit we use for sampling)
    assert not isinstance(base_model, TransformerWithValueHead), "Please pass in the base model, not the model wrapper."

    # Convert our prompt into tokens
    input_ids = base_model.to_tokens(prompt, prepend_bos=False).squeeze(0)

    # Generate samples (we repeat the input ids which is a bit wasteful but ¯\_(ツ)_/¯)
    input_ids = einops.repeat(input_ids, "seq -> batch seq", batch=batch_size)
    # input_ids = einops.repeat(input_iget_logprobsds, "seq -> batch seq", batch=batch_size)

    # Generate samples
    output_ids = base_model.generate(
        input_ids,
        max_new_tokens = gen_len,
        stop_at_eos = False,
        temperature = temperature, # higher means more random completions
        verbose = False,
    )
    samples = base_model.to_string(output_ids)

    return output_ids.clone(), samples

Here's some example use of this function:

In [51]:
sample_ids, samples = get_samples(
    model.base_model,
    prompt = "So long, and thanks for all the",
    batch_size = 5,
    gen_len = 20,
    temperature = 0.8,
)

table = Table("Token IDs", "Samples", title="Demo of `sample` function", show_lines=True)

for ids, sample in zip(sample_ids, samples):
    table.add_row(str(ids.tolist()), repr(sample))

rprint(table)

                                             Demo of `sample` function                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Token IDs                                              ┃ Samples                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 7237, 13,    │ "So long, and thanks for all the emails. I've been     │
│ 314, 1053, 587, 8179, 351, 21740, 290, 1972, 2641,     │ busy with prizes and getting inside information of     │
│ 1321, 286, 6041, 286, 1180, 22830, 13, 314, 1053]      │ lots of different contests. I've"                      │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 1393, 13,    │ "So long, and thanks for all the                       │
│ 50256, 1026, 1422, 470, 1011, 502, 890, 284, 651, 284, │ interest.<|endoftext|>It didn't take me long to get to │
│ 670, 357, 392, 314, 1101, 9675, 314, 750]              │ work (and I'm glad I did"                              │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 1037, 290,   │ 'So long, and thanks for all the help and              │
│ 3725, 0, 198, 198, 14126, 718, 25, 854, 27148, 198,    │ knowledge!\n\nChapter 6: Whispers\n\n"Oh… everyone\'s  │
│ 198, 1, 5812, 1399, 2506, 338, 587, 523]               │ been so'                                               │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 1257, 0,     │ 'So long, and thanks for all the fun!                  │
│ 7359, 79, 29, 50256, 4711, 49109, 15956, 300, 14124,   │ </p><|endoftext|>These pesky zombie lizards are so     │
│ 389, 523, 25129, 1775, 319, 428, 5440, 13, 1119]       │ seldom seen on this planet. They'                      │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 1243, 0,     │ 'So long, and thanks for all the things!\n\nMake sure  │
│ 198, 198, 12050, 1654, 284, 1061, 502, 319, 17044,     │ to follow me on twitter :)                             │
│ 14373, 1303, 2436, 6201, 7959, 50256, 14698, 1257,     │ #blazedev<|endoftext|>Having fun undertaking'          │
│ 25971]                                                 │                                                        │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

### Exercise - implement `reward_fn_char_count`

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend ~5 minutes on this exercise.
```

We'll start with a very basic reward function: counting the total number of periods in the sequence. For convenience, you should write your reward function to take in either a single sequence or a list of sequences (it will correspondingly return either a float or a tensor of floats).


An interesting thing to note about this reward function - it counts over all characters, but the episode length is defined in terms of tokens. This means that theoretically our model could reward hack by outputting tokens with more than one `.` character. This particular model's vocabulary happens to include the token `'.' * 64`, so rewards would be through the roof if this was ever generated! However, remember that RL is about performing actions, getting feedback on those actions, and using that feedback to influence your policy. The token `'.' * 64` is so unlikely to ever be generated that it'll probably never be positively reinforced, and we avoid this problem.

In [52]:
import torch as t
from torch import Tensor
from typing import Union, List
from jaxtyping import Float

def reward_fn_char_count(generated_sample: Union[str, List[str]], char: str = '.') -> Union[float, Float[Tensor, "batch"]]:
    '''
    Reward function, evaluated on the generated samples.

    In this case it's very simple: it just counts the number of instances of a particular character in
    the generated sample. It returns a tensor of rewards of dtype float the input is a list, or a single
    reward (float) if the input is a string.
    '''
    if isinstance(generated_sample, list):
        return t.tensor([reward_fn_char_count(item) for item in generated_sample]).float().to("cuda")
    else:
        return float(generated_sample.count(char))


# Test your reward function
A = 'This is a test.'
B = '......'
C = 'Whatever'
assert isinstance(reward_fn_char_count(A), float)
assert reward_fn_char_count(A) == 1
assert reward_fn_char_count(B) == 6
assert reward_fn_char_count(C) == 0
assert reward_fn_char_count([A, B, C]).dtype == t.float
assert reward_fn_char_count([A, B, C]).tolist() == [1.0, 6.0, 0.0]

print('All tests for `reward_fn_char_count` passed!')

All tests for `reward_fn_char_count` passed!


### Exercise - brainstorm your reward function

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend ~5 minutes on this exercise.
```

Take 5 minutes (on your own or with a partner) to brainstorm how the model might be able to maximize the output of periods in ways which don't produce incoherent output (e.g. collapsing into only outputting periods). Remember we have a KL penalty with the reference model, meaning the model is penalized for producing outputs which would be very unlikely under the original model. What ideas can you come up with? When you train your model and observe the output, you should come back here and see how many of the period-maximizing behaviours you predicted actually occur.

This exercise is a great way to start thinking about the effects of different reward functions - although it's only a toy example, it still illustrates the important alignment concept that the behaviour induced by certain reward functions might not always be what you expect!

<details>
<summary>Spoiler - which behaviours will your model pick up?</summary>

The strategies adopted by the model very a lot depending on the prefix string, also thanks to mode collapse it will often find one of these behaviours and entirely ignore the others.

Some common strategies include:

- Shorter sentences
- Repeating `U.S.` or `U.S.A.` (using the prefix prompt `"There is"`, this seems to be by far the most common strategy)
- Library versions e.g. `Python 2.7.12` or `the 2.6.0.2 release`
- Names with initials e.g. `C. S. Lewis` or titles e.g. `Dr.` and `PhD.`
- Abbreviations e.g. `Data-R.A.R. series` or `"L.A. Times"`
- Decimals in numbers e.g. `9.5cm x 7.5 cm`
- Triple periods e.g. `the man . . . the woman . . .`

</details>

### Exercise - implement `normalize_reward`

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend ~5 minutes on this exercise.
```

Following advice from Ziegler el al. (2019), it's important to normalize the reward function over each batch (i.e. subtract mean and divide by std dev). We've been able to get away with not doing this so far because our reward functions were usually nicely bounded, e.g. the reward was always zero or one in cartpole (and even in our reward shaping it was still in the zero-one range). But if we're working with reward functions that could be much higher variance such as the number of periods in a generated sequence, then we should normalize.

Note - we're not super strict about this function; the denominator being `std + eps` or `(var + eps).sqrt()` are both fine.

In [54]:
def normalize_reward(reward: Float[Tensor, "batch_size"], eps=1e-5) -> Float[Tensor, "batch_size"]:
    '''
    Normalizes the reward function values over the batch of sequences.
    '''
    return (reward - reward.mean()) / (reward.std() + eps)


# Test your reward normalization function
reward = 10 + 5 * t.randn(10_000)
reward_normalized = normalize_reward(reward)
assert reward_normalized.mean().abs() < 1e-4
assert (reward_normalized.std() - 1).abs() < 1e-4
# Test edge case of zero reward
reward = t.zeros(5)
reward_normalized = normalize_reward(reward)
assert reward_normalized.abs().sum() < 1e-4

print('All tests for `normalize_reward` passed!')

All tests for `normalize_reward` passed!


## RLHF training args

We've given you a bunch of training args to use for RLHF. Most of these are similar to the PPO args, although we'll go over the ones we've added / removed.

- We're now using `total_phases` to control how long our training lasts for, rather than using `total_timesteps`. This makes more sense for us, because the total number of timesteps (= number of actions we take = number of tokens we generate) will vary depending on the length of the sequences we generate.
- We've removed the arguments `gamma` and `gae_lambda` for computing the advantage function, since as discussed we'll be computing the advantage in a simpler and more direct way (you'll do this in the next exercise).
- We've added the following arguments related to the base model & text sampling:
    - `base_model`, for specifying different base models (default is `"gpt2-small"`)
    - `gen_len`, the length of the sequences we generate.
    - `temperature`, for controlling the sampling temperature of our sequences.
    - `prefix`, the string we use to generate all samples.
- As well as the following extra RLHF-specific arguments:
    - `kl_coef`, for controlling the strength of the KL prediction shift penalty.
    - `reward_fn`, for the reward function we use.
    - `normalize_reward`, for whether we normalize the reward (this won't always be necessary).
- We've also added two learning rates, since it makes sense to have a different learning rate for our value head and the rest of the model (more on this later!).

In [55]:
@dataclass
class RLHFTrainingArgs():

    # Basic / global
    seed: int = 1
    cuda: bool = t.cuda.is_available()

    # Wandb / logging
    exp_name: str = "RLHF_Implementation"
    wandb_project_name: Optional[str] = "ch2-day4-rlhf"
    wandb_entity: Optional[str] = None
    use_wandb: bool = False

    # Duration of different phases
    total_phases: int = 200
    batch_size: int = 256
    num_minibatches: int = 4
    batches_per_learning_phase: int = 2

    # Optimization hyperparameters
    base_learning_rate: float = 2e-5
    head_learning_rate: float = 5e-4
    max_grad_norm: float = 1.0
    warmup_steps: int = 20
    final_scale: float = 0.1

    # Computing other PPO loss functions
    clip_coef: float = 0.2
    vf_coef: float = 0.15
    ent_coef: float = 0.001

    # Base model & sampling arguments
    base_model: str = "gpt2-medium"
    gen_len: int = 30
    temperature: float = 0.6
    prefix: str = "This is"

    # Extra stuff for RLHF
    kl_coef: float = 1.0
    reward_fn: Callable = reward_fn_char_count
    normalize_reward: bool = True

    def __post_init__(self):
        assert self.batch_size % self.num_minibatches == 0, "Batch size should be divisible by the number of minibatches."
        self.minibatch_size = self.batch_size // self.num_minibatches

### Exercise - implement `get_advantages`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-20 minutes on this exercise.
```

As we discussed earlier, your advantage function doesn't need to use GAE like yesterday. Instead, we'll base our estimates on the simple formula:

$$
A(s_t, a_t) = Q(s_t, a_t) - V(s_t)
$$

In place of $Q(s_t, a_t)$ we'll use the **one-step Q estimates**, i.e. our value function estimates after taking action $a_t$ at step $s_t$, meaning we're at new state $s_{t+1} = [s_t \; a_t]$. This is just the same as $V(s_{t+1})$ for all values of $t$ except at the last sequence position, since we can replace this value estimate with the known reward $r_t$ for the whole sequence.

The baseline $V(s_t)$ terms will just be using our **zero-step value estimates**.

We want to compute the advantages for every action taken after the prefix prompt, i.e. our output will have shape `[minibatch_size, gen_length]` where `gen_length` is the number of generated tokens from above. Note - your values tensor will have shape `[minibatch_size, gen_length + prefix_len]`, since it contains the value estimates for the whole sequence (including the prefix prompt). Make sure you slice correctly when computing the advantages!

This diagram should help explain things:

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/rlhf-advantages-2.png" width="900">

In [77]:
tt = t.zeros(1)
tt.all

In [83]:
@t.no_grad()
def compute_advantages(
    values: Float[Tensor, "minibatch_size seq_len"],
    rewards: Float[Tensor, "minibatch_size"],
    prefix_len: int,
) -> Float[Tensor, "minibatch_size gen_len"]:
    '''
    Computes the advantages for the PPO loss function, i.e. A_pi(s, a) = Q_pi(s, a) - V_pi(s).

    In this formula we replace Q(s, a) with the 1-step Q estimates, and V(s) with the 0-step value estimates.

    Inputs:
        values:
            the value estimates for each token in the generated sequence
        rewards:
            the rewards for the entire generated sequence
        prefix_len:
            the length of the prefix (i.e. the length of the initial prompt)

    Returns:
        advantages:
            the advantages for each token in the generated sequence (not the entire sequence)
    '''
    # print(f'{rewards.shape=} {values.shape=} {rewards.unsqueeze(-1).shape=}')
    Q = t.cat((values[:, prefix_len: -1], rewards.unsqueeze(-1)), dim=-1)
    V = values[:, prefix_len-1: -1]
    # print(f'{Q.shape=} {V.shape=}')
    return Q - V


tests.test_compute_advantages(compute_advantages)

All tests in `test_compute_advantages` passed!


## Memory

We've given you an implementation of the `ReplayMemory` and `ReplayMinibatch` classes.

Some notes on how `ReplayMinibatch` differs from the PPO implementation:

- We don't need to store `actions` any more, because the actions (tokens generated) are in contained within the sequences themselves.
- We don't need to store `dones` any more, because all our sequences last for exactly `gen_length` steps.
- We need to store `ref_logits`, which are used to compute the KL penalty with respect to our reference model.

Some notes on how `ReplayMemory` differs from the PPO implementation:

- We can afford to make it a lot simpler. We don't need to worry about sampling minibatches, and we also don't need to worry about having methods to add experiences one by one (we'll add them all at once).
- We've computed `returns` (i.e. advantages + values) for only the positions where we'll generate new sequences (because we only evaluated advantages at these sequence positions). Later functions will include some error checking to make sure your tensors are the right shape (although it's good to add this kind of error checking yourself too!).
- We still use roughly the same structure for creating minibatches: for each `batches_per_learning_phase`, we generate a set of `num_minibatches` minibatches, with each of these minibatches being disjoint but having union equal to the whole batch. This means that every sequence will be seen exactly `batches_per_learning_phase` times per learning phase, just like we had in PPO yesterday.

In [84]:
@dataclass
class ReplayMinibatch:
    '''
    Samples from the replay memory.
    '''
    sample_ids: Float[Tensor, "minibatch_size seq_len"]
    logprobs: Float[Tensor, "minibatch_size seq_len"]
    advantages: Float[Tensor, "minibatch_size gen_len"]
    returns: Float[Tensor, "minibatch_size gen_len"]
    ref_logits: Float[Tensor, "minibatch_size seq_len d_vocab"]


class ReplayMemory:
    def __init__(
        self,
        args: RLHFTrainingArgs,
        sample_ids: Float[Tensor, "batch_size seq_len"],
        logprobs: Float[Tensor, "batch_size seq_len"],
        advantages: Float[Tensor, "batch_size gen_len"],
        values: Float[Tensor, "batch_size seq_len"],
        ref_logits: Float[Tensor, "batch_size seq_len d_vocab"],
    ):
        '''
        Initializes the replay memory, with all the data generated from the rollout phase at once.

        The advantages are (batch_size, gen_len) because we only compute advantages for the generated
        tokens. The other tensors are (batch_size, seq_len) because they are computed for all tokens.
        '''
        self.args = args
        self.sample_ids = sample_ids
        self.logprobs = logprobs
        self.advantages = advantages
        self.values = values
        self.ref_logits = ref_logits


    def get_minibatches(self) -> List[ReplayMinibatch]:
        '''
        Generates a list of minibatches by randomly sampling from the replay memory. Each sequence appears
        exactly `batches_per_learning_phase` times in total.
        '''
        minibatches = []

        returns = self.advantages + self.values[:, -self.args.gen_len-1:-1]

        for _ in range(self.args.batches_per_learning_phase):

            idxs = t.randperm(self.args.batch_size).reshape(self.args.num_minibatches, self.args.minibatch_size)

            for idx in idxs:
                minibatches.append(
                    ReplayMinibatch(
                        sample_ids = self.sample_ids[idx],
                        logprobs = self.logprobs[idx],
                        advantages = self.advantages[idx],
                        returns = returns[idx],
                        ref_logits = self.ref_logits[idx],
                    )
                )

        return minibatches

## Objective function

### Exercise - implement `calc_kl_penalty`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

Now, you'll implement the KL penalty function. As discussed, the purpose of this function is to make sure your new model doesn't diverge too much from the old model. We'll be using the KL divergence between the old and new models' logit distributions.

The formula for KL divergence of two distributions $(P, Q)$ is $\sum_i P_i \log (P_i / Q_i)$. Recall that we want our new logits to be $P$ and reference logits to be $Q$ (because this penalizes our new model for generating outputs which would be very unlikely under the original reference model).

You should take the mean over batch and sequence position dims, since each token represents a separate observation and action.

There are a couple more important points to address, before implementing this function:



## OUR GROUP Asside on entropy
$entropy = - \sum P_i log(P_i)$

In [89]:
-(0.9*np.log(0.9) + 0.1 * np.log(0.1))

0.3250829733914482

In [88]:
-(0.5*np.log(0.5) + 0.5 * np.log(0.5))

0.6931471805599453

In [93]:
0.5 * np.log(0.5/(1/3))

0.2027325540540822

In [90]:
0.25 * np.log(0.25/(1/3)) + 0.25 * np.log(0.25/(1/3))

-0.14384103622589045

In [92]:
0.3 * np.log(0.3/(1/3)) + 0.2 * np.log(0.2/(1/3))

-0.13377327945054598

#### Prefix tokens vs generated tokens

**We only want to compute KL for the new tokens generated.** This means we want to look at the logits for the tokens at sequence positions `prefix_len-1:-1`, since these correspond to **predictions for tokens after the prefix, within the generated sequence**. There's no point penalizing the model for having a different probability distribution on the tokens prefix prompt, since this won't directly induce a behavioural change (our prefix prompt is always fixed). We only care about changing the model's behaviour on the actual tokens it generates.



#### Numerical stability

There can be a problem when dealing with extreme values, since operations like log / exp are numerically unstable and can give you NaN values. To avoid this, make sure your order of operations is correct. **You should always start with `log_softmax` to compute logprobs, then take `exp` to compute probabilities,** and avoid functions `log` and `softmax`. This is because `log_softmax` subtracts a constant from all the values to avoid any of them being extremely large when we calculate the necessary exponentials (recall that the softmax operation is translation invariant i.e. we can add the same value to all logits and the probs / logprobs are still the same). We can then take `exp` of the result to get probabilities because taking `exp` of negative values is numerically stable. On the other hand, taking `softmax` to get (possibly very small) probabilities and then `log` to get (possibly very extreme negative) logprobs is not numerically stable.

In [118]:
def calc_kl_penalty(
    logits: Float[Tensor, "minibatch_size seq_len d_vocab"],
    ref_logits: Float[Tensor, "minibatch_size seq_len d_vocab"],
    kl_coef: float,
    prefix_len: int,
) -> Float[Tensor, ""]:
    '''
    Computes the KL divergence between the logits and the reference logits, scaled
    by the penalty function. This is used to stop the learned policy from diverging
    too much from the original reference model's policy.

    logits:
        The logits of the generated samples (under the new model).
    ref_logits:
        The logits of the generated samples (under the reference model).
    kl_coef:
        The coefficient of the KL penalty.
    prefix_len:
        The length of the prefix to ignore when computing the KL divergence.
    '''
    log_l = logits.log_softmax(-1)[:, prefix_len-1: -1]
    log_ref = ref_logits.log_softmax(-1)[:, prefix_len-1: -1]
    kl = (log_l.exp() * (log_l - log_ref)).sum(-1)
    return kl.mean() * kl_coef

tests.test_calc_kl_penalty(calc_kl_penalty)
tests.test_calc_kl_penalty_stability(calc_kl_penalty)

All tests in `test_calc_kl_penalty` passed!
All tests in `test_calc_kl_penalty_stability` passed!


### Exercise - (re)implement `compute_entropy_bonus`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to ~10 minutes on this exercise.
```

Next, we'll implement the entropy bonus function again. Rather than working with `probs.entropy()` like yesterday, we'll need to compute entropy directly from the logits, and take the mean over batch and sequence position dimensions.

The formula for entropy of a distribution $P$ is $- \sum_i P_i \log P_i$.

As for the previous exercise, you'll need to slice the logits to only look at the new tokens generated. Also, you'll need to take the same numerical stability precautions.

In [126]:
def calc_entropy_bonus(
    logits: Float[Tensor, "minibatch_size seq_len d_vocab"],
    ent_coef: float,
    prefix_len: int
) -> Float[Tensor, ""]:
    '''
    Return the entropy bonus term, suitable for gradient ascent.

    logits:
        the logits of the tokens generated by the model.
    ent_coef:
        the coefficient for the entropy loss, which weights its contribution to the overall objective function.
    prefix_len:
        The length of the prefix to ignore when computing the KL divergence.
    '''
    log_l = logits[:, prefix_len-1: -1].log_softmax(-1)
    entropy = - (log_l.exp() * log_l).sum(-1)
    return entropy.mean() * ent_coef


tests.test_calc_entropy_bonus(calc_entropy_bonus)
tests.test_calc_entropy_bonus_stability(calc_entropy_bonus)

All tests in `test_calc_entropy_bonus` passed!
All tests in `test_calc_entropy_bonus_stability` passed!


### Other objective function terms

Since the other two terms in our objective function (value function loss and clipped surrogate objective) are pretty much identical to yesterday's, we've provided them for you (taken from yesterday's solutions code).

Note that **these functions all only take input of shape `gen_len`** (because advantages and returns are created with shape `gen_len`). So we don't need to do any positional slicing in these functions, like for the last two. When it comes to using these loss functions.

In [127]:
def calc_value_function_loss(
    values: Float[Tensor, "minibatch_size gen_len"],
    mb_returns: Float[Tensor, "minibatch_size gen_len"],
    vf_coef: float
) -> Float[Tensor, ""]:
    '''Compute the value function portion of the loss function.

    values:
        the value function predictions for the sampled minibatch (using the updated critic network)
    mb_returns:
        the target for our updated critic network (computed as `advantages + values` from the old network)
    vf_coef:
        the coefficient for the value loss, which weights its contribution to the overall loss. Denoted by c_1 in the paper.
    '''
    assert values.shape == mb_returns.shape,\
        f"Shape mismatch: {values.shape=}, {mb_returns.shape=}. Did you slice 'values' tokens correctly?"
    return 0.5 * vf_coef * (values - mb_returns).pow(2).mean()


def calc_clipped_surrogate_objective(
    logprobs: Float[Tensor, "minibatch_size gen_len"],
    mb_logprobs: Float[Tensor, "minibatch_size gen_len"],
    mb_advantages: Float[Tensor, "minibatch_size gen_len"],
    clip_coef: float,
    eps: float = 1e-8,
) -> Float[Tensor, ""]:
    '''Return the clipped surrogate objective, suitable for maximisation with gradient ascent.

    logprobs:
        the logprobs of the action taken by the agent, according to the new policy
    mb_logprobs:
        logprobs of the actions taken in the sampled minibatch (according to the old policy)
    mb_advantages:
        advantages calculated from the sampled minibatch
    clip_coef:
        amount of clipping, denoted by epsilon in Eq 7.
    eps:
        used to add to std dev of mb_advantages when normalizing (to avoid dividing by zero)
    '''
    assert logprobs.shape == mb_logprobs.shape == mb_advantages.shape,\
        f"Shape mismatch: {logprobs.shape=}, {mb_logprobs.shape=}, {mb_advantages.shape=}. Did you create logprobs using 'get_logprobs' correctly?"

    logits_diff = logprobs - mb_logprobs

    r_theta = t.exp(logits_diff)

    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + eps)

    non_clipped = r_theta * mb_advantages
    clipped = t.clip(r_theta, 1-clip_coef, 1+clip_coef) * mb_advantages

    return t.minimum(non_clipped, clipped).mean()

### Exercise - implement `get_logprobs`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

You'll notice that the functions above take logprobs of shape `(minibatch_size, gen_len)`, i.e. the logprobs on correct tokens for all the tokens generated by the model. It'll be useful to implement the following function (which should be pretty familiar to you by now).

This function takes `logits` and the corresponding `tokens` which were used as input, and returns an array of logprobs for the correct next tokens. Note, we're only taking the logits which are predicting tokens **after the prefix**, hence we've given you a `prefix_len` argument for this function. The diagram below should help explain this. Note, you can assume `prefix_len` will always be at least 1.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/get-correct-logprobs-3-solid.png" width="520">

You can implement this function using regular indexing, tools like `torch.gather`, or with the `eindex` library which should be included in your dependencies (see [here](https://www.perfectlynormal.co.uk/blog-eindex) for how to use this library).

In [ ]:
tokens = [[10, 20, 15]     0
         ,[17, 2, 110]]    1
           0   1   2

In [149]:
def get_logprobs(
    logits: Float[Tensor, "batch seq_len vocab"],
    tokens: Int[Tensor, "batch seq_len"],
    prefix_len: int = 1,
) -> Float[Tensor, "batch gen_len"]:
    '''
    Returns correct logprobs for the given logits and tokens, for all the tokens
    after the prefix tokens (which have length equal to `prefix_len`).

    If prefix_len = None then we return shape (batch, seq_len-1). If not, then
    we return shape (batch, seq_len-prefix_len) representing the predictions for
    all tokens after the prefix tokens.
    '''
    prefix_len = prefix_len or 1
    log_l = logits[:, prefix_len-1:-1].log_softmax(-1)
    tokens = tokens[:, prefix_len:]
    # print(f'{log_l.shape=} {tokens.shape=}')

    # eindex
    ret = correct_logprobs = eindex(log_l, tokens, "batch seq [batch seq] -> batch seq")

    # loop
    # ret = [][]
    # for batch in range(..):
    #   for seq in range(..):
    #     ret[batch, seq] = log_l[batch, seq, tokens[batch, seq]]

    # vectorized
    # batch_indices = np.arange(log_l.shape[0])
    # seq_indices = np.arange(log_l.shape[1])
    # ret = log_l[batch_indices[:, None], seq_indices, tokens]

    # gather
    # ret = t.gather(log_l, -1, tokens[:,:,None])[:,:,0]
    return ret


tests.test_get_logprobs(get_logprobs)

All tests for `get_logprobs` passed (for prefix_len = None)!
All tests for `get_logprobs` passed (for prefix_len > 0)!


## Optimizer & Scheduler

### Exercise - implement `get_optimizer`

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

We need to be a bit careful when defining our optimizer. It makes no sense to have the same learning rate for our original model as we do for our value head. The value head was randomly initialized and has no idea what it's doing, but our model is pretrained and so it already has weights which have been trained to effectively extract features from text.

The syntax for using parameter groups in an optimizer is as follows:

```python
parameter_groups = [
    {"params": [param1, param2, ...], "lr": lr1},
    {"params": [param3, param4, ...], "lr": lr2},
]
```

where `params` is a list (or iterable) of parameters, and `lr` is the learning rate for these parameters.

You should fill in the function `get_optimizer` below, so that the value head's parameters all have learning rate `args.head_learning_rate` and the base model's parameters all have learning rate `args.base_learning_rate`.

Remember that we're using `maximize=True` with our optimizer (since we're maximizing an objective function rather than minimizing a loss function).

In [153]:
def get_optimizer(args: RLHFTrainingArgs, model: TransformerWithValueHead) -> t.optim.Optimizer:
    '''
    Returns an Adam optimizer for the model, with the correct learning rates for the base and head.
    '''
    return t.optim.Adam([
      {"params": model.base_model.parameters(), "lr": args.base_learning_rate},
      {"params": model.value_head.parameters(), "lr": args.head_learning_rate},
    ], maximize=True)


args = RLHFTrainingArgs()
optimizer = get_optimizer(args, model)

assert len(optimizer.param_groups) == 2, "Your optimizer should have two parameter groups."

for param_group in optimizer.param_groups:
    assert param_group["maximize"] == True, "Should be maximize=True."
    if len(param_group["params"]) <= 4:
        assert param_group["lr"] == args.head_learning_rate, "LR for value head should be `args.head_learning_rate`."
    else:
        assert param_group["lr"] == args.base_learning_rate, "LR for base should be `args.base_learning_rate`."

total_params = sum(len(param_group["params"]) for param_group in optimizer.param_groups)
assert total_params == len(list(model.parameters())), "Your optimizer should have the same number of parameters as the model."

print("All tests for `get_optimizer` passed!")

All tests for `get_optimizer` passed!


### Scheduler

In PPO, we had you write a custom class for implementing learning rate scheduling. This was useful to help you engage with the nuts and bolts of changing learning rates in Pytorch. However, PyTorch does provide a handy class for implementing custom learning rate scheduling. The syntax looks like this:

```python
optimizer = t.optim.Adam(...)
scheduler = t.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
```

where `lr_lambda` is a function which takes in an integer (the total number of times `scheduler.step()` has been called) and returns a float (which **gets multiplied by the base learning rate** to create the new learning rate). Note that there are also schedulers other than `LambdaLR` which have specific behaviour and so you wouldn't need to write custom functions for them - you can read about these midway down [this documentation page](https://pytorch.org/docs/stable/optim.html).

We've given you the code you'll be using for returning a custom `lr_lambda` function with a **linear warmup then linear decay**. We've also provided code for you in the trainer class's init method below which creates your scheduler. All you need to do is make sure you're stepping it appropriately.

<br>

<details>
<summary>Aside - why we use warmup</summary>

Warmup is a common strategy early in training, to make sure we don't get excessive updates early on. It seems to work pretty well empirically. Some possible reasons for this are:

* It helps avoid large updates when the Adam moving averages of first and second moments are not yet well calibrated.
* Early on in training, the gradients might be very large (especially for the value function) because the model's prediction is nowhere near where it needs to be. So an LR warmup is more useful early on, to help avoid massive steps.

</details>

<br>

Note - yesterday we stepped our scheduler after every optimizer step. But it's more common practice in PyTorch to step the scheduler once every epoch. In this case, we'll be stepping it **at the end of each training phase**.

In [154]:
def get_lr_scheduler(warmup_steps, total_steps, final_scale):
    '''
    Creates an LR scheduler that linearly warms up for `warmup_steps` steps,
    and then linearly decays to `final_scale` over the remaining steps.
    '''
    def lr_lambda(step):
        assert step <= total_steps, f"Step = {step} should be less than total_steps = {total_steps}."
        if step < warmup_steps:
            return step / warmup_steps
        else:
            return 1 - (1 - final_scale) * (step - warmup_steps) / (total_steps - warmup_steps)

    return lr_lambda


def get_optimizer_and_scheduler(args: RLHFTrainingArgs, model: TransformerWithValueHead):
    optimizer = get_optimizer(args, model)
    lr_lambda = get_lr_scheduler(args.warmup_steps, args.total_phases, args.final_scale)
    scheduler = t.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
    return optimizer, scheduler

If we want to log the learning rate, then we can use `scheduler.get_last_lr()[0]`.

## Training your model

We're now ready to put everything together! We've provided you with the skeleton of a training loop which should be very similar to yesterday's.



### Exercise - complete `RLHFTrainer`

```c
Difficulty: 🔴🔴🔴🔴🔴
Importance: 🔵🔵🔵🔵🔵

You should spend up to 40-60 minutes on this exercise.
```

The functions you need to fill in have docstrings which should help. A few bits of guidance:

- Make sure the shapes of objects you are passing into functions is correct - especially when it comes to slicing objects of shape `(batch_size, seq_len)` across the `seq_len` dimension to remove the prefix prompt (or not).
- For faster feedback loops, don't use `wandb` until you've stopped getting errors!
- For an easy test of whether your model is working, use the hyperparameter `kl_coef=0.0` and print your generated sequences; you should quickly see the model collapse into saying `"This is......"`.

#### Logging to wandb - recap

If you want to log text to Weights & Biases, there are 2 main ways:

1. Just print output, this is logged to weights & biases under the "Logs" section!
2. Log tables. This should usually be done just once at the end of training (because you can't log tables incrementally, only all at once). Here's some example code I used here for logging all my samples in a single table, as well as my hyperparameters (useful when creating a run report):

```python
wandb.log({
    "samples_table": wandb.Table(["sample"], self.samples),
    "config_params": wandb.Table(["param", "values"], [[k, v.__name__ if callable(v) else str(v)] for k, v in self.args.__dict__.items()])
})
```

This works when `self.samples` is a list of length-1 lists, each containing a single sample (i.e. one of the strings returned frmo the `get_samples` method).

In [155]:
class RLHFTrainer:
    model: TransformerWithValueHead
    ref_model: HookedTransformer
    memory: ReplayMemory # we'll set this during rollout

    def __init__(self, args: RLHFTrainingArgs):
        t.manual_seed(args.seed)
        self.args = args
        self.run_name = f"{args.exp_name}__{args.seed}__{int(time.time())}"
        self.model = TransformerWithValueHead(args.base_model).to(device).train()
        self.ref_model = TransformerWithValueHead(args.base_model).to(device).eval()
        self.optimizer, self.scheduler = get_optimizer_and_scheduler(self.args, self.model)
        self.prefix_len = len(self.model.base_model.to_str_tokens(self.args.prefix, prepend_bos=False))


    def compute_rlhf_objective(self, mb: ReplayMinibatch):
        '''
        Computes the RLHF objective function to maximize, which equals the PPO objective function minus
        the KL penalty term.

        Steps of this function are:
            - Get logits & values for the samples in minibatch
            - Get the logprobs of the minibatch actions taken
            - Use this data to compute all 4 terms of the RLHF objective function, and create function
        '''
        logits, values = self.model(mb.sample_ids)
        values = values[:, self.prefix_len-1:-1]

        # Get logprobs for the the tokens generated (i.e. the logprobs of our actions)
        logprobs = get_logprobs(logits, mb.sample_ids, self.prefix_len)

        # Compute all terms of the loss function (including KL penalty)
        # TODO ask Changlin between this clip and the grad clip, why is it not redundant
        clipped_surrogate_objective = calc_clipped_surrogate_objective(logprobs, mb.logprobs, mb.advantages, self.args.clip_coef)
        value_loss = calc_value_function_loss(values, mb.returns, self.args.vf_coef)
        entropy_bonus = calc_entropy_bonus(logits, self.args.ent_coef, self.prefix_len)
        kl_penalty = calc_kl_penalty(logits, mb.ref_logits, self.args.kl_coef, self.prefix_len)

        # Compute net objective function
        ppo_objective_fn = clipped_surrogate_objective - value_loss + entropy_bonus
        total_objective_function = ppo_objective_fn - kl_penalty

        return total_objective_function


    def rollout_phase(self) -> ReplayMemory:
        '''
        Performs a single rollout phase, retyrning a ReplayMemory object containing the data generated
        during this phase. Note that all forward passes here should be done in inference mode.

        Steps of this function are:
            - Generate samples from our model
            - Get logits of those generated samples (from model & reference model)
            - Get other data for memory (logprobs, normalized rewards, advantages)
            - Return this data in a ReplayMemory object
        '''
        # Get our samples
        sample_ids, samples = get_samples(
            self.model.base_model,
            prompt=self.args.prefix,
            batch_size=self.args.batch_size,
            gen_len=self.args.gen_len,
            temperature=self.args.temperature,
        )

        # Generate logits from our model & reference model
        with t.inference_mode():
            logits, values = self.model(sample_ids)
            ref_logits = self.ref_model(sample_ids)

        logprobs = get_logprobs(logits, sample_ids, self.prefix_len)

        # Calculate & normalize rewards
        rewards = self.args.reward_fn(samples)
        if self.args.normalize_reward: rewards = normalize_reward(rewards)

        # Compute advantages
        advantages = compute_advantages(values, rewards, self.prefix_len)

        return ReplayMemory(
            args = self.args,
            sample_ids = sample_ids,
            logprobs = logprobs,
            advantages = advantages,
            values = values,
            ref_logits = ref_logits,
        )



    def learning_phase(self, memory: ReplayMemory) -> None:
        '''
        Performs a learning step on `self.memory`. This involves the standard gradient descent steps
        (i.e. zeroing gradient, computing objective function, doing backprop, stepping optimizer).

        You should also remember the following:
            - Clipping grad norm to the value given in `self.args.max_grad_norm`
            - Incrementing `self.step` by 1 for each phase (not minibatch!)
            - Stepping the scheduler (once per calling of this function)
        '''
        for minibatch in memory.get_minibatches():
            self.optimizer.zero_grad()
            loss = self.compute_rlhf_objective(minibatch)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=self.args.max_grad_norm)
            self.optimizer.step()
            self.step += 1
        self.scheduler.step()

    def train(self) -> None:
        '''
        Performs a full training run.
        '''
        self.step = 0

        if self.args.use_wandb: wandb.init(
            project = self.args.wandb_project_name,
            entity = self.args.wandb_entity,
            name = self.run_name,
            config = self.args,
        )

        for self.phase in range(self.args.total_phases):
            memory = self.rollout_phase()
            self.learning_phase(memory)

        if self.args.use_wandb: wandb.finish()

## version with wandb

In [157]:
class RLHFTrainer:
    model: TransformerWithValueHead
    ref_model: HookedTransformer
    memory: ReplayMemory # we'll set this during rollout

    def __init__(self, args: RLHFTrainingArgs):

        t.manual_seed(args.seed)
        self.args = args
        self.run_name = f"{args.exp_name}__{args.seed}__{int(time.time())}"
        self.model = TransformerWithValueHead(args.base_model).to(device).train()
        self.ref_model = HookedTransformer.from_pretrained(args.base_model).to(device).eval()
        self.optimizer, self.scheduler = get_optimizer_and_scheduler(self.args, self.model)
        self.prefix_len = len(self.model.base_model.to_str_tokens(self.args.prefix, prepend_bos=False))


    def compute_rlhf_objective(self, mb: ReplayMinibatch):
        '''
        Computes the RLHF objective function to maximize, which equals the PPO objective function minus
        the KL penalty term.

        Steps of this function are:
            - Get logits & values for the samples in minibatch
            - Get the logprobs of the minibatch actions taken
            - Use this data to compute all 4 terms of the RLHF objective function, and create function
        '''

        # Get logits & values for our generated minibatch samples, and index values cause we never need all of them
        logits, values = self.model(mb.sample_ids)
        values = values[:, self.prefix_len-1:-1]

        # Get logprobs for the the tokens generated (i.e. the logprobs of our actions)
        logprobs = get_logprobs(logits, mb.sample_ids, self.prefix_len)

        # Compute all terms of the loss function (including KL penalty)
        clipped_surrogate_objective = calc_clipped_surrogate_objective(logprobs, mb.logprobs, mb.advantages, self.args.clip_coef)
        value_loss = calc_value_function_loss(values, mb.returns, self.args.vf_coef)
        entropy_bonus = calc_entropy_bonus(logits, self.args.ent_coef, self.prefix_len)
        kl_penalty = calc_kl_penalty(logits, mb.ref_logits, self.args.kl_coef, self.prefix_len)

        # Compute net objective function
        ppo_objective_fn = clipped_surrogate_objective - value_loss + entropy_bonus
        total_objective_function = ppo_objective_fn - kl_penalty

        # Log stuff
        with t.inference_mode():
            logratio = logprobs - mb.logprobs
            ratio = logratio.exp()
            clipfracs = [((ratio - 1.0).abs() > self.args.clip_coef).float().mean().item()]
        if self.args.use_wandb: wandb.log(dict(
            total_steps = self.step,
            learning_rate = self.scheduler.get_last_lr()[0],
            clipped_surrogate_objective = clipped_surrogate_objective.item(),
            clipfrac = np.mean(clipfracs),
            value_loss = value_loss.item(),
            values = values.mean().item(),
            entropy_bonus = entropy_bonus.item(),
            kl_penalty = kl_penalty.item(),
        ), step=self.step)

        return total_objective_function


    def rollout_phase(self) -> ReplayMemory:
        '''
        Performs a single rollout phase, retyrning a ReplayMemory object containing the data generated
        during this phase. Note that all forward passes here should be done in inference mode.

        Steps of this function are:
            - Generate samples from our model
            - Get logits of those generated samples (from model & reference model)
            - Get other data for memory (logprobs, normalized rewards, advantages)
            - Return this data in a ReplayMemory object
        '''
        # Get our samples
        sample_ids, samples = get_samples(
            self.model.base_model,
            prompt=self.args.prefix,
            batch_size=self.args.batch_size,
            gen_len=self.args.gen_len,
            temperature=self.args.temperature,
        )
        self.samples.append([samples[0]])

        # Generate logits from our model & reference model
        with t.inference_mode():
            logits, values = self.model(sample_ids)
            ref_logits = self.ref_model(sample_ids)

        logprobs = get_logprobs(logits, sample_ids, self.prefix_len)

        # Calculate & normalize rewards
        rewards = self.args.reward_fn(samples)
        rewards_mean = rewards.mean().item()
        if self.args.normalize_reward: rewards = normalize_reward(rewards)

        # Compute advantages
        advantages = compute_advantages(values, rewards, self.prefix_len)

        # Log stuff, and print output in a readable way
        if self.args.use_wandb: wandb.log({'mean_reward': rewards_mean}, step=self.step)

        # # Print output (simple version)
        # print(f"  Mean reward: {rewards_mean:.4f}\n  Sample: {samples[0]!r}")

        # Print output (nice version, using rich print)
        ref_logprobs = get_logprobs(ref_logits[:3], sample_ids[:3], self.prefix_len).sum(-1)
        table = Table(
            "Reward", "Ref logprobs", "Sample",
            title=f"Phase {self.phase:03}/{self.args.total_phases}, Mean reward: {rewards_mean:.4f}",
            show_lines=True
        )
        for r, lp, s in zip(rewards.tolist(), ref_logprobs, samples):
            table.add_row(str(int(r)), f"{lp:.2f}", repr(s))
        rprint(table); print("")

        return ReplayMemory(
            args = self.args,
            sample_ids = sample_ids,
            logprobs = logprobs,
            advantages = advantages,
            values = values,
            ref_logits = ref_logits,
        )


    def learning_phase(self, memory: ReplayMemory) -> None:
        '''
        Performs a learning step on `self.memory`. This involves computing
        '''
        for minibatch in memory.get_minibatches():
            self.optimizer.zero_grad()
            loss = self.compute_rlhf_objective(minibatch)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=self.args.max_grad_norm)
            self.optimizer.step()
            self.step += 1

        self.scheduler.step()


    def train(self) -> None:
        '''
        Performs a full training run.
        '''
        self.step = 0
        self.samples = []

        if self.args.use_wandb: wandb.init(
            project = self.args.wandb_project_name,
            entity = self.args.wandb_entity,
            name = self.run_name,
            config = self.args,
        )

        for self.phase in range(self.args.total_phases):
            memory = self.rollout_phase()
            self.learning_phase(memory)

        if self.args.use_wandb:
            wandb.log({
                "samples_table": wandb.Table(["sample"], self.samples),
                "config_params": wandb.Table(["param", "values"], [[k, v.__name__ if callable(v) else str(v)] for k, v in self.args.__dict__.items()])
            })
            wandb.finish()

<details>
<summary>Solution (simple, no logging)</summary>

```python
class RLHFTrainer:
    model: TransformerWithValueHead
    ref_model: HookeTransformer
    memory: ReplayMemory # we'll set this during rollout

    def __init__(self, args: RLHFTrainingArgs):

        t.manual_seed(args.seed)
        self.args = args
        self.run_name = f"{args.exp_name}__{args.seed}__{int(time.time())}"
        self.model = TransformerWithValueHead(args.base_model).to(device).train()
        self.ref_model = HookedTransformer.from_pretrained(args.base_model).to(device).eval()
        self.optimizer, self.scheduler = get_optimizer_and_scheduler(self.args, self.model)
        self.prefix_len = len(self.model.base_model.to_str_tokens(self.args.prefix, prepend_bos=False))


    def compute_rlhf_objective(self, mb: ReplayMinibatch):
        '''
        Computes the RLHF objective function to maximize, which equals the PPO objective function minus
        the KL penalty term.

        Steps of this function are:
            - Get logits & values for the samples in minibatch
            - Get the logprobs of the minibatch actions taken
            - Use this data to compute all 4 terms of the RLHF objective function, and create function
        '''
        # Get logits & values for our generated minibatch samples, and index values cause we never need all of them
        logits, values = self.model(mb.sample_ids)
        values = values[:, self.prefix_len-1:-1]

        # Get logprobs for the the tokens generated (i.e. the logprobs of our actions)
        logprobs = get_logprobs(logits, mb.sample_ids, self.prefix_len)

        # Compute all terms of the loss function (including KL penalty)
        clipped_surrogate_objective = calc_clipped_surrogate_objective(logprobs, mb.logprobs, mb.advantages, self.args.clip_coef)
        value_loss = calc_value_function_loss(values, mb.returns, self.args.vf_coef)
        entropy_bonus = calc_entropy_bonus(logits, self.args.ent_coef, self.prefix_len)
        kl_penalty = calc_kl_penalty(logits, mb.ref_logits, self.args.kl_coef, self.prefix_len)

        # Compute net objective function
        ppo_objective_fn = clipped_surrogate_objective - value_loss + entropy_bonus
        total_objective_function = ppo_objective_fn - kl_penalty

        return total_objective_function
    

    def rollout_phase(self) -> ReplayMemory:
        '''
        Performs a single rollout phase, retyrning a ReplayMemory object containing the data generated
        during this phase. Note that all forward passes here should be done in inference mode.

        Steps of this function are:
            - Generate samples from our model
            - Get logits of those generated samples (from model & reference model)
            - Get other data for memory (logprobs, normalized rewards, advantages)
            - Return this data in a ReplayMemory object
        '''
        # Get our samples
        sample_ids, samples = get_samples(
            self.model.base_model,
            prompt=self.args.prefix,
            batch_size=self.args.batch_size,
            gen_len=self.args.gen_len,
            temperature=self.args.temperature,
        )

        # Generate logits from our model & reference model
        with t.inference_mode():
            logits, values = self.model(sample_ids)
            ref_logits = self.ref_model(sample_ids)
        
        logprobs = get_logprobs(logits, sample_ids, self.prefix_len)

        # Calculate & normalize rewards
        rewards = self.args.reward_fn(samples)
        if self.args.normalize_reward: rewards = normalize_reward(rewards)

        # Compute advantages
        advantages = compute_advantages(values, rewards, self.prefix_len)

        return ReplayMemory(
            args = self.args,
            sample_ids = sample_ids,
            logprobs = logprobs,
            advantages = advantages,
            values = values,
            ref_logits = ref_logits,
        )
    

    def learning_phase(self, memory: ReplayMemory) -> None:
        '''
        Performs a learning step on `self.memory`. This involves computing
        '''
        for minibatch in memory.get_minibatches():
            self.optimizer.zero_grad()
            loss = self.compute_rlhf_objective(minibatch)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)
            self.optimizer.step()
            self.step += 1
        
        self.scheduler.step()
    

    def train(self) -> None:
        '''
        Performs a full training run.
        '''
        self.step = 0
        
        if self.args.use_wandb: wandb.init(
            project = self.args.wandb_project_name,
            entity = self.args.wandb_entity,
            name = self.run_name,
            config = self.args,
        )
        
        for self.phase in range(self.args.total_phases):
            memory = self.rollout_phase()
            self.learning_phase(memory)

        if self.args.use_wandb: wandb.finish()
```

</details>

<details>
<summary>Solution (full logging)</summary>

```python
class RLHFTrainer:
    model: TransformerWithValueHead
    ref_model: HookedTransformer
    memory: ReplayMemory # we'll set this during rollout

    def __init__(self, args: RLHFTrainingArgs):

        t.manual_seed(args.seed)
        self.args = args
        self.run_name = f"{args.exp_name}__{args.seed}__{int(time.time())}"
        self.model = TransformerWithValueHead(args.base_model).to(device).train()
        self.ref_model = HookedTransformer.from_pretrained(args.base_model).to(device).eval()
        self.optimizer, self.scheduler = get_optimizer_and_scheduler(self.args, self.model)
        self.prefix_len = len(self.model.base_model.to_str_tokens(self.args.prefix, prepend_bos=False))


    def compute_rlhf_objective(self, mb: ReplayMinibatch):
        '''
        Computes the RLHF objective function to maximize, which equals the PPO objective function minus
        the KL penalty term.

        Steps of this function are:
            - Get logits & values for the samples in minibatch
            - Get the logprobs of the minibatch actions taken
            - Use this data to compute all 4 terms of the RLHF objective function, and create function
        '''

        # Get logits & values for our generated minibatch samples, and index values cause we never need all of them
        logits, values = self.model(mb.sample_ids)
        values = values[:, self.prefix_len-1:-1]

        # Get logprobs for the the tokens generated (i.e. the logprobs of our actions)
        logprobs = get_logprobs(logits, mb.sample_ids, self.prefix_len)

        # Compute all terms of the loss function (including KL penalty)
        clipped_surrogate_objective = calc_clipped_surrogate_objective(logprobs, mb.logprobs, mb.advantages, self.args.clip_coef)
        value_loss = calc_value_function_loss(values, mb.returns, self.args.vf_coef)
        entropy_bonus = calc_entropy_bonus(logits, self.args.ent_coef, self.prefix_len)
        kl_penalty = calc_kl_penalty(logits, mb.ref_logits, self.args.kl_coef, self.prefix_len)

        # Compute net objective function
        ppo_objective_fn = clipped_surrogate_objective - value_loss + entropy_bonus
        total_objective_function = ppo_objective_fn - kl_penalty

        # Log stuff
        with t.inference_mode():
            logratio = logprobs - mb.logprobs
            ratio = logratio.exp()
            clipfracs = [((ratio - 1.0).abs() > self.args.clip_coef).float().mean().item()]
        if self.args.use_wandb: wandb.log(dict(
            total_steps = self.step,
            learning_rate = self.scheduler.get_last_lr()[0],
            clipped_surrogate_objective = clipped_surrogate_objective.item(),
            clipfrac = np.mean(clipfracs),
            value_loss = value_loss.item(),
            values = values.mean().item(),
            entropy_bonus = entropy_bonus.item(),
            kl_penalty = kl_penalty.item(),
        ), step=self.step)

        return total_objective_function
    

    def rollout_phase(self) -> ReplayMemory:
        '''
        Performs a single rollout phase, retyrning a ReplayMemory object containing the data generated
        during this phase. Note that all forward passes here should be done in inference mode.

        Steps of this function are:
            - Generate samples from our model
            - Get logits of those generated samples (from model & reference model)
            - Get other data for memory (logprobs, normalized rewards, advantages)
            - Return this data in a ReplayMemory object
        '''
        # Get our samples
        sample_ids, samples = get_samples(
            self.model.base_model,
            prompt=self.args.prefix,
            batch_size=self.args.batch_size,
            gen_len=self.args.gen_len,
            temperature=self.args.temperature,
        )
        self.samples.append([samples[0]])

        # Generate logits from our model & reference model
        with t.inference_mode():
            logits, values = self.model(sample_ids)
            ref_logits = self.ref_model(sample_ids)
        
        logprobs = get_logprobs(logits, sample_ids, self.prefix_len)

        # Calculate & normalize rewards
        rewards = self.args.reward_fn(samples)
        rewards_mean = rewards.mean().item()
        if self.args.normalize_reward: rewards = normalize_reward(rewards)

        # Compute advantages
        advantages = compute_advantages(values, rewards, self.prefix_len)

        # Log stuff, and print output in a readable way
        if self.args.use_wandb: wandb.log({'mean_reward': rewards_mean}, step=self.step)
        table = Table("Reward", "Sample", title=f"Phase {self.phase:03}/{self.args.total_phases}, Mean reward: {rewards_mean:.4f}", show_lines=True)
        for r, s in zip(rewards.tolist()[:3], samples):
            table.add_row(str(int(r)), repr(s))
        rprint(table); print("")

        return ReplayMemory(
            args = self.args,
            sample_ids = sample_ids,
            logprobs = logprobs,
            advantages = advantages,
            values = values,
            ref_logits = ref_logits,
        )
    

    def learning_phase(self, memory: ReplayMemory) -> None:
        '''
        Performs a learning step on `self.memory`. This involves computing
        '''
        for minibatch in memory.get_minibatches():
            self.optimizer.zero_grad()
            loss = self.compute_rlhf_objective(minibatch)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)
            self.optimizer.step()
            self.step += 1
        
        self.scheduler.step()
    

    def train(self) -> None:
        '''
        Performs a full training run.
        '''
        self.step = 0
        self.samples = []
        
        if self.args.use_wandb: wandb.init(
            project = self.args.wandb_project_name,
            entity = self.args.wandb_entity,
            name = self.run_name,
            config = self.args,
        )
        
        for self.phase in range(self.args.total_phases):
            memory = self.rollout_phase()
            self.learning_phase(memory)

        if self.args.use_wandb:
            wandb.log({
                "samples_table": wandb.Table(["sample"], self.samples),
                "config_params": wandb.Table(["param", "values"], [[k, v.__name__ if callable(v) else str(v)] for k, v in self.args.__dict__.items()])
            })
            wandb.finish()
```

</details>

Once you've implemented your trainer class, you can run the code below to train your model. You can also play around with the parameters - in particular, try a few different prefix strings. The behaviour of the model (e.g. which kinds of techniques it converges onto for period maximization) or whether it easily mode collapses into insanity can be highly dependent on the prefix string!

Some common strategies you should observe include:

- Shorter sentences
- Repeating `U.S.` or `U.S.A.` (using the prefix prompt `"There is"`, this seems to be by far the most common strategy)
- Library versions e.g. `Python 2.7.12` or `the 2.6.0.2 release`
- Names with initials e.g. `C. S. Lewis` or titles e.g. `Dr.` and `PhD.`
- Abbreviations e.g. `Data-R.A.R. series` or `"L.A. Times"`
- Decimals in numbers e.g. `9.5cm x 7.5 cm`
- Triple periods e.g. `the man . . . the woman . . .`

You might also observe increasingly incoherent mode collapse if you train for too long. Here are a few that I got:

- `This is really helpful. The U.S. U.S. U.S. U.S.`
- `This is the A.A.G.A.R.M.A.R.M.A.R.M.A.R.M`
- `This is my mother. . . me. . . . . . . . . . . . . . . . . . . . . . . .`

[Here's](https://api.wandb.ai/links/callum-mcdougall/es12ygmm) an example run, which was generated using the default parameters (i.e. the code below).

In [158]:
args = RLHFTrainingArgs(use_wandb=True)
trainer = RLHFTrainer(args)
trainer.train()

Loaded pretrained model gpt2-medium into HookedTransformer
Loaded pretrained model gpt2-medium into HookedTransformer
Moving model to device:  cuda


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

                                        Phase 196/200, Mean reward: 6.7031                                         
┏━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Reward ┃ Ref logprobs ┃ Sample                                                                                  ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0      │ -50.25       │ 'This is not necessarily a good thing. The U.S. is not a rich country. The U.S. economy │
│        │              │ is not booming. The U.'                                                                 │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -59.78       │ 'This is probably why the U.S. has a higher per capita income than most other rich      │
│        │              │ countries. The United States has a large economy. The U.'                               │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -50.46       │ 'This is the difference between the U.S. and the British. The U.S. is a capitalist      │
│        │              │ country. The British are not capitalists. The U'                                        │
└────────┴──────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

                                        Phase 197/200, Mean reward: 6.6836                                         
┏━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Reward ┃ Ref logprobs ┃ Sample                                                                                  ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0      │ -55.34       │ 'This is what the U.S. has learned. The U.S. has learned that the U.S. will always have │
│        │              │ a military of the United'                                                               │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -59.64       │ 'This is not the case, and the mistake was made. The U.S. is not the problem. The U.S.  │
│        │              │ is not the problem.'                                                                    │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -56.12       │ "This is not a joke. This is real. This is the truth. This is what's going on. The U.S. │
│        │              │ military is killing babies."                                                            │
└────────┴──────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

                                        Phase 198/200, Mean reward: 6.9492                                         
┏━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Reward ┃ Ref logprobs ┃ Sample                                                                                  ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0      │ -54.33       │ 'This is how the U.S. has been doing it for decades. The U.S. is a global superpower.   │
│        │              │ The U.S. is economically'                                                               │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -44.33       │ 'This is not the way the United States was built. The U.S. was built on the backs of    │
│        │              │ workers. The U.S. was built on'                                                         │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -55.10       │ 'This is a big step forward for the U.S. in terms of the international community. The   │
│        │              │ U.S. has a role to play. We are'                                                        │
└────────┴──────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

                                        Phase 199/200, Mean reward: 7.0391                                         
┏━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Reward ┃ Ref logprobs ┃ Sample                                                                                  ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0      │ -60.86       │ 'This is not a joke. It\'s real."\n\nThe U.S. is not alone in having difficulty holding │
│        │              │ the line. The U.K.'                                                                     │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -61.37       │ 'This is what the Internet is about. The U.S. is not. If the U.S. is not a democracy,   │
│        │              │ the U.S.'                                                                               │
├────────┼──────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ 0      │ -58.77       │ 'This is why the U.S. military will not tolerate the U.N. Security Council. The U.S.    │
│        │              │ military is the United Nations. It'                                                     │
└────────┴──────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

clipfrac,▁▂▅▅▇▅▅▇▅▆▇▆▆▇█▆█▅▆▇▄▅▆▅▄▅▅▆▄▄▄▃▄▅▃▂▂▂▂▁
clipped_surrogate_objective,▁▂▅▅▆▆▆▇▆▆▅▇▇▇██▇▇▆▇▆▅▅▆▆▇▆▅▅▅▄▄▄▅▃▄▅▂▃▁
entropy_bonus,███▅▇▆▆▆▅▄▃▆▃▃▆▄▄▂▂▂▅▂▃▅▄▃▁▃▂▃▁▂▃▄▃▂▂▂▁▁
kl_penalty,▁▁▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▆▆▇▇█▇▇▇▇▇▇▇█
learning_rate,▁▂▅▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁
mean_reward,▁▁▂▂▂▂▂▂▃▃▄▃▄▄▄▄▅▄▅▆▄▆▅▄▅▆▆▆▅▆▇▇▆▇▇▇▇▇██
total_steps,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
value_loss,▇▃▅▇▇▆█▅▆▆▇▃▅▁▄▃▃▄▄▂▇▂▄▅▁▂▁▅▂▂▂▃▂▂▁▂▂▄▃▂
values,▃▂▂▅██▇▆▆▇▇▅▆▅▄▅▄▄▄▅▂▄▃▁▂▃▆▃▁▃▃▄▂▂▃▂▃▂▄▄
clipfrac,0.01198
clipped_surrogate_objective,0.01482


Example params for gpt2-large (I think this would work a lot better with layer freezing but haven't tried it, see bonus exercises!).

In [ ]:
# args = RLHFTrainingArgs(use_wandb=True, base_model="gpt2-large", num_minibatches=16, kl_coef=0.5)
# trainer = RLHFTrainer(args)
# trainer.train()

### Exercise - use a more complex reward function

```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 30-50 minutes on this exercise.
```


We recommend you experiment with a few different reward functions, in particular some sentiment-based reward functions which are based on pretrained text classification models. For example, you might want to start with one of the following:

- [`lvwerra/distilbert-imdb`](https://huggingface.co/lvwerra/distilbert-imdb), which was trained to classify IMDB film reviews as positive or negative.
- [`cardiffnlp/twitter-roberta-base-sentiment`](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment), which is a model trained on tweets and finetuned for sentiment analysis (categories are positive, neutral and negative).
- [`distilbert-base-uncased-emotion`](bhadresh-savani/distilbert-base-uncased-emotion), which was finetuned on the [Emotion Dataset for Emotion Recognition Tasks](https://www.kaggle.com/datasets/parulpandey/emotion-dataset), i.e. it's trained to classify text according to emotional tone (classes are sadness, joy, love, anger, fear and surprise).

Note that for some of these, you should be using a prompt string which is appropriate for the reward function you're fine-tuning on, e.g. `"This movie was really"` for the IMDB model. Similarly, you might also want to change other parameters e.g. generation length. You can find a list of other models [here](https://huggingface.co/models?filter=text-classification).

The solutions contain the following:

- A reward function derived from the `lvwerra/distilbert-imdb` model (with demo of it working),
- A wandb run of GPT2-medium using this reward function (both in positive and negative sentiment directions).

We recommend you try to do something similar before you compare your answers to the solutions.

In [ ]:
# YOUR CODE HERE - define a reward function derived from a sentiment model (and sanity
# check it by printing out some outputs on some chosen sequences).

In [ ]:
# YOUR CODE HERE - train your model on this reward function

For reference, you can see the parameters & results of a positive-sentiment IMDB run [here](https://api.wandb.ai/links/callum-mcdougall/3a1bl3y4), and a negative run [here](https://api.wandb.ai/links/callum-mcdougall/misa79ct).

# 2️⃣ Bonus

## Extensions of today's RLHF exercises

### Large models

We're already working with `gpt2-medium` which is considerably larger than most of the models you worked with in most of the transformers & interpretability material. Can you go even larger?

See [this page](https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html) for a table of model properties, for all models currently supported by TransformerLens. Note that if you use different model classes then you might need to change some parts of your code (e.g. if the name of the hook point where you added the value head happens to be different). You might also need to make other adjustments e.g. a smaller batch size (or a larger number of minibatches per batch, which is equivalent to smaller minibatch sizes).

<br>

### Differential Learning Rates / Frozen Layers

When doing any kind of finetuning, it's common practice to either freeze earlier layers or have a smaller learning rate for them. You may have seen this in the feature extraction with ResNet34 exercises in the first week. In the exercises here we've trained all layers of the model equally, but you might want to play around with differential learning rates.

Note that you can accomplish this using parameter groups - we already used parameter groups above to have a different learning rate for our base model and value head. It should be relatively straightforward to extend this to splitting parameters over different layers into different groups (hint - you can use `itertools.chain` to convert several iterables into a single iterable).

You can also try entirely freezing earlier layers - this might also reduce your memory usage, and allow you to train larger models without getting cuda errors.

<br>

### Hyperparameter sweeps

You can do this to find the best possible hyperparamters for your RLHF training. Don't just measure on reward, can you use some combination of reward and avg kl diff to create a better metric? Can you use wandb's built-in [Bayesian search methods](https://docs.wandb.ai/guides/sweeps/sweep-config-keys#bayesian-search) to more effectively sweep?

Note - don't forget **temperature** when it comes to hyperparameter tuning. Temperature has an important effect on how the model learns, e.g. if the temperature is too high then the model will produce very high-variance outputs which will have very high KL with the reference distribution, and it'll be more likely to collapse into some incoherent mode.

<br>

### Adaptive KL penalty

The KL divergence penalty coefficient can be modified adaptively based on the KL divergence between the current policy and the previous policy. If the KL divergence is outside a predefined target range, we can adjust the penalty coefficient to bring it closer to the target range. Here is an example implementation:

```python
class AdaptiveKLController:
    def __init__(self, init_kl_coef, hparams):
        self.value = init_kl_coef
        self.hparams = hparams

    def update(self, current, n_steps):
        target = self.hparams.target
        proportional_error = np.clip(current / target - 1, -0.2, 0.2)
        mult = 1 + proportional_error * n_steps / self.hparams.horizon
        self.value *= mult
```

<br>

### TRL / trlX

We've been focusing on building RLHF from the ground up, but there are several libraries which exist to abstract away manuy of the low-level implementation details we had to wrestle with. One of the best-known is TRL (Transformer Reinforcement Learning). The main docs page can be found [here](https://huggingface.co/docs/trl/index), and [this page](https://huggingface.co/docs/trl/quickstart) gives a quickstart guide. You may find it much easier to use this library than to implement everything yourself!

Read their documentation pages, and see what techniques they use to make RLHF more effective. Are there any that we haven't implemented here? Can you implement them yourself?

You might also be interested in trlX, an expanded fork of TRL built by CarperAI to handle larger models for online and offline training (although their APIs are pretty similar).

<br>

### Learn a human preference reward model

We've been working with a pre-supplied reward function, but you can try and train your own!

We'll give some brief points of guidance here, for the task of training a reward function on the **summarization task**. Note that these instructions have been provided externally, so they've not yet been tested and might not work particularly well.

1. Get a supervised baseline
    * [Here](https://zenodo.org/records/1168855) is a link to download the dataset for the TL;DR challenge containing posts from the Reddit corpus. Each post contains keys `content` and `summary` which are the original post and the human-written summary respectively.
    * You should throw out all summaries shorter than 24 tokens or longer than 48 tokens (to diminish the effects of length on quality); and choose a random subset of ~100k summaries to train on.
    * Run training to maximize the log-likelihood of these summaries.
2. Get reward model by training supervised baseline on human feedback
    * Download comparison data with the code `azcopy copy "https://openaipublic.blob.core.windows.net/summarize-from-feedback/dataset/*" . --recursive`
    * Modify GPT-2 architecture by adding a randomly-initialized **reward head** at the end of your model.
        * Architecturally this is similar to the value head from earlier, but it's not the same thing - here we're trying to learn what the human reward will be; we're not doing RL yet.
    * Train your model (starting with base model given by supervised baseline weights, and reward head randomly initialized) to minimize `loss = log(sigmoid(reward_model(summary_0) - reward_model(summary_1)))`, `summary_0` is preferred by a human labeler (this data should be in the comparison data you downloaded).
    * You should normalize reward model outputs, like we normalized rewards in RLHF in previous exercises.
3. Fine-tune supervised baseline using PPO with reward model.
    * For these exercises we suggest using a larger model, ideally GPT2-Large or bigger. Remember you can freeze weights! Regardless, this will still take longer to train than your previous models.

<br>

### Interp on RLHF'd models

Currently, very little mechanistic interpretability research ahs focused on RLHF'd models. In [this blog post](https://blog.eleuther.ai/trlx-exploratory-analysis/), Curt Tigges walks through an example of how we can use mech interp to analyze a model which has been finetuned with a sentiment based reward function using trlX.

The flavour of the actual mech interp done here is very similar to the indirect object identification exercises you might have done during the transformers & interp week. If you didn't do these exercises, we recommend you do them before diving deep into this material.

Lastly, here's a [Google Doc](https://docs.google.com/document/d/1eUdvlJNqY9X0NAw9UUseZz6dFyRklCcOHQy8x3CbcBk/edit?usp=sharing) brainstorming some ideas for RLHF interpretability. You might find some ideas there (although most of these will be pretty vague goals so possibly too ambitious for a bonus exercise or 1-week project).

## Suggested paper replications

As well as the three papers in this section, you might be interested in browsing this [GitHub repo](https://github.com/opendilab/awesome-RLHF), which contains links to a large number of RLHF-related papers.

### [Deep Reinforcement Learning from Human Preferences](https://arxiv.org/abs/1706.03741)

This was the seminal paper in RLHF. They applied it to the domain of tasks like MuJoCo (which you might already have worked with during your PPO day). Can you set up a reward function and an interface which allows you to choose between two different sets of trajectories, and learn a reward function to maximize?

Some more technical details here - the authors train the reward function at the same time as they train the model. In other words, after a certain number of iterations of (rollout phase, learning phase), they add a third reward model learning phase, where the current policy generates many pairs of trajectories of some fixed timestep and the human rater chooses which one is best. They famously trained the Hopper agent to perform repeated backflips using just 900 queries.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/hopper-backflip.png" width="700">

[Here](https://drive.google.com/drive/folders/0BwcFziBYuA8RM2NTdllSNVNTWTg?resourcekey=0-w4PuSuFvi3odgQXdBDPQ0g) is the link mentioned in the image caption.

Note - we strongly recommend doing the PPO exercises on MuJoCo before attempting this replication. We also recommend using Colab, since MuJoCo is notoriously difficult to install all the dependencies for!

<br>

### [Measuring Faithfulness in Chain-of-Thought Reasoning](https://arxiv.org/abs/2307.13702)

This paper investigates the **chain-of-thought prompting method**, by examining how model predictions change when this chain is intervened on. They find that models show large variation across tasks in how strongly they condition on the CoT when predicting their answer, sometimes relying heavily on the CoT and other times primarily ignoring it.

<br>

### [Recursively Summarizing Books with Human Feedback](https://arxiv.org/abs/2109.10862)

A major challenge for scaling ML is training models to perform tasks that are very difficult or time-consuming for humans to evaluate. To test scalable alignment techniques, the authors trained a model to summarize entire books, by first summarizing small sections of a book, then summarizing those summaries into a higher-level summary, and so on. A demonstration can be found [here](https://openai.com/research/summarizing-books). There is also a [repository](https://github.com/openai/summarize-from-feedback) containing code to run their models, including the supervised baseline, the trained reward model, and the RL fine tuned policy.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/alice.png" width="500">

You may also wish to do this in a less directed way - see the bonus exercise “Learn a human preference reward model” above.